# NCI WeatherBench-3d: Train a CNN (PyTroch)

In this notebook we will go through all the steps required to train a fully convolutional neural network.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/g/data/wb00/admin/testing/NCI_weatherbench/NCI_notebooks_v3/src')

import os
import warnings
warnings.filterwarnings('ignore')
from score import *
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from datetime import datetime
from dask.distributed import Client

import torch
import torch.nn as nn
import torch.nn.functional as F
import train_nn_pytorch

In [3]:
client = Client(n_workers=12, threads_per_worker=1)  
client

/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.events-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/opt/conda/envs/mlenv/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.wa

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43743,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 95.00 GiB
Comm: tcp://127.0.0.1:44163,Total threads: 1
Dashboard: /proxy/42007/status,Memory: 7.92 GiB
Nanny: tcp://127.0.0.1:34643,


# Dataset: '5.625'

In [4]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

res     = '5.625'
datadir = f'/g/data/wb00/NCI-Weatherbench/{res}deg' 
print ("Data loading..." )

years = list(range(1999, 2022+1))
print (years)
z_files = [ file for year in years for file in glob.glob (fr'{datadir}/geopotential/*{year}*')  ] 
t_files = [ file for year in years for file in glob.glob (fr'{datadir}/temperature/*{year}*')    ] 

z = xr.open_mfdataset(z_files, combine='by_coords', parallel=True, chunks={'time': 10}).z.sel(level=[500]).load() 
t = xr.open_mfdataset(t_files, combine='by_coords', parallel=True, chunks={'time': 10}).t.sel(level=[850]).drop('level').load() 
datasets = [z, t]
print ("Merging ... ")
ds = xr.merge(datasets).compute()  

[2024-04-21 21:49:43]
Data loading...
[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Merging ... 
CPU times: user 1min 44s, sys: 10.2 s, total: 1min 54s
Wall time: 2min 27s


In [5]:
z['time'][:10]

<xarray.DataArray 'time' (time: 10)>
array(['1999-01-01T00:00:00.000000000', '1999-01-01T01:00:00.000000000',
       '1999-01-01T02:00:00.000000000', '1999-01-01T03:00:00.000000000',
       '1999-01-01T04:00:00.000000000', '1999-01-01T05:00:00.000000000',
       '1999-01-01T06:00:00.000000000', '1999-01-01T07:00:00.000000000',
       '1999-01-01T08:00:00.000000000', '1999-01-01T09:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1999-01-01 ... 1999-01-01T09:00:00

In [6]:
z['time'][-10:]

<xarray.DataArray 'time' (time: 10)>
array(['2022-12-31T14:00:00.000000000', '2022-12-31T15:00:00.000000000',
       '2022-12-31T16:00:00.000000000', '2022-12-31T17:00:00.000000000',
       '2022-12-31T18:00:00.000000000', '2022-12-31T19:00:00.000000000',
       '2022-12-31T20:00:00.000000000', '2022-12-31T21:00:00.000000000',
       '2022-12-31T22:00:00.000000000', '2022-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-12-31T14:00:00 ... 2022-12-31T23:00:00

In [7]:
def data_generate(ds, lead_time, batch_size, variables, train_years, valid_years, test_years):
    ds_train = ds.sel(time=slice(*train_years))
    ds_valid = ds.sel(time=slice(*valid_years))
    ds_test  = ds.sel(time=slice(*test_years))

    print ("Data generation ... ")
    dic = {var: 500 for var in variables} #vars}
    dg_train = train_nn_pytorch.DataGenerator(ds_train, dic, lead_time, batch_size=batch_size)
    dg_valid = train_nn_pytorch.DataGenerator(ds_valid, dic, lead_time, batch_size=batch_size, mean=dg_train.mean,
                         std=dg_train.std, shuffle=False)
    dg_test =  train_nn_pytorch.DataGenerator(ds_test, dic, lead_time, batch_size=batch_size, mean=dg_train.mean,
                         std=dg_train.std, shuffle=False)

    print(f'Mean = {dg_train.mean}; Std = {dg_train.std}')
    return dg_train, dg_valid, dg_test

def train(dg_train_generator, dg_valid_generator, dg_test_generator, model_save_fn):
    print ("Train model ... ") 
    
    model = train_nn_pytorch.Model_cnn(channels, kernels)
    
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    print('cuda' if torch.cuda.is_available() else 'cpu')
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)    
    
    patience = n_patience
    best_loss = float('inf')

    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = []     

    for epoch in range(n_epochs):
        for X, y in dg_train_generator:
            model.train()
            X  = torch.squeeze( X )
            X = X.to(device)
            y_pred = model (X)  
            y  = torch.squeeze( y )
            y = y.to(device)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            for X_v, y_v in dg_valid_generator:
                X_v  = torch.squeeze( X_v )
                y_v  = torch.squeeze( y_v )
                X_v = X_v.to(device)
                y_v = y_v.to(device)

                y_v_pred = model(X_v)
                loss_v = loss_fn(y_v_pred, y_v)
                valid_losses.append(loss_v.item())

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        epoch_len = len(str(n_epochs))

        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')

        print(print_msg)

        # clear  
        train_losses = []
        valid_losses = []  

        if loss_v < best_loss:  
            best_loss = loss_v
            patience = n_patience  
        else:
            patience -= 1
            if patience == 0:
                print(f'Early stopping')
                break               

    return model  
        
def evaluate(pred, iterative, test_years):     
    print("Evaluating forecast ...")
    valid_years = list( range(int(test_years[0]), int(test_years[1])+1 ))
    print ('all test_years:', valid_years)
    z500_valid_files = [ file for year in valid_years for file in glob.glob (fr'{datadir}/geopotential/*{year}*') ] 
    t850_valid_files = [ file for year in valid_years for file in glob.glob (fr'{datadir}/temperature/*{year}*')  ]     

    z500_valid = load_test_data(z500_valid_files, 'z', slice(*test_years)) 
    t850_valid = load_test_data(t850_valid_files, 't', slice(*test_years))     
    
    valid      = xr.merge([z500_valid, t850_valid], compat='override').compute()
    
    print(train_nn_pytorch.evaluate_iterative_forecast(pred, valid, compute_weighted_rmse).load() if iterative \
                                                else train_nn_pytorch.compute_weighted_rmse(pred, valid).compute())


In [8]:
test_years =('2021', '2022')
test_years = list (range(int (test_years[0]), int(test_years[1])+1 ))
print ('test_years:', test_years)

test_years: [2021, 2022]


# 72 hours (3 days)

In [9]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
print (60*"-")
train_years=('1999', '2015')
valid_years=('2016', '2020')
test_years =('2021', '2022')
batch_size = 32
variables = ('z', 't')
lead_time = 72
channels = [2, 64, 64, 64, 64, 64, 2]
kernels = [5, 5, 5, 5, 5, 5]
dr = 0
save_prefix = 'PyTorch_NCI_tutorial' 
print ('save_prefix:', save_prefix)
model_save_fn = f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/{save_prefix}_cnn_3d.h5'
pred_save_fn  = f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/{save_prefix}_cnn_3d.nc'
lr = 1e-4
iterative = False
iterative_lead_time = None
n_epochs = 100   
n_patience = 4

[2024-04-21 21:52:10]
------------------------------------------------------------
save_prefix: PyTorch_NCI_tutorial
CPU times: user 0 ns, sys: 64 µs, total: 64 µs
Wall time: 55.6 µs


In [10]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
dg_train, dg_valid, dg_test = data_generate(ds, lead_time, batch_size,
                                           variables, train_years, valid_years, test_years)

params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 6}
dg_train_generator = torch.utils.data.DataLoader(dg_train, **params)
params['shuffle'] = False
dg_valid_generator = torch.utils.data.DataLoader(dg_valid, **params)
dg_test_generator = torch.utils.data.DataLoader(dg_test, **params)

model = train(dg_train_generator, dg_valid_generator, dg_test_generator, model_save_fn)

print(f'Saving model weights: {model_save_fn}')
torch.save(model.state_dict(), model_save_fn)

[2024-04-21 21:52:10]
Data generation ... 
Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54140.027  ,   274.65103], dtype=float32)
Coordinates:
  * level    (level) int32 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1100.1848  ,    5.642098], dtype=float32)
Coordinates:
  * level    (level) int32 500 500
Train model ... 
cuda
[  0/100] train_loss: 0.52872 valid_loss: 0.46290
[  1/100] train_loss: 0.43961 valid_loss: 0.43917
[  2/100] train_loss: 0.41554 valid_loss: 0.42754
[  3/100] train_loss: 0.39897 valid_loss: 0.42233
[  4/100] train_loss: 0.38638 valid_loss: 0.41749
[  5/100] train_loss: 0.37584 valid_loss: 0.41576
Early stopping
Saving model weights: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/PyTorch_NCI_tutorial_cnn_3d.h5
CPU times: user 11min 25s, sys: 1min 1s, total: 12min 26s
Wall time: 12min 3s


In [11]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
pred = train_nn_pytorch.create_predictions(model, dg_test_generator, 
                        mean=dg_train.mean.values, std=dg_train.std.values,
                        var_dict=dg_test.var_dict,
                        valid_time=dg_test.valid_time, 
                        lat=dg_test.ds.lat, 
                        lon=dg_test.ds.lon 
                       )
print(f'Saving predictions: {pred_save_fn}')
pred.to_netcdf(pred_save_fn)

evaluate(pred, iterative, test_years)

[2024-04-21 22:04:14]
cuda
torch.Size([17448, 2, 32, 64])
Saving predictions: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/PyTorch_NCI_tutorial_cnn_3d.nc
Evaluating forecast ...
all test_years: [2021, 2022]
load_test_data, var: z
load_test_data, var: t
<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 643.1
    t        float64 2.96
CPU times: user 11.6 s, sys: 2.39 s, total: 14 s
Wall time: 17.1 s


# 120 hours (5 days)

In [12]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

print (60*"-")
batch_size = 32
lead_time = 120
print ('save_prefix:', save_prefix)
model_save_fn= f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/{save_prefix}_cnn_5d.h5'
pred_save_fn = f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/{save_prefix}_cnn_5d.nc'
lr = 1e-4
iterative = False
iterative_lead_time = None

[2024-04-21 22:04:31]
------------------------------------------------------------
save_prefix: PyTorch_NCI_tutorial
CPU times: user 0 ns, sys: 58 µs, total: 58 µs
Wall time: 54.4 µs


In [13]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

dg_train, dg_valid, dg_test = data_generate(ds, lead_time, batch_size,
                                           variables, train_years, valid_years, test_years)

params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 6}
dg_train_generator = torch.utils.data.DataLoader(dg_train, **params)
params['shuffle'] = False
dg_valid_generator = torch.utils.data.DataLoader(dg_valid, **params)
dg_test_generator = torch.utils.data.DataLoader(dg_test, **params)

model = train(dg_train_generator, dg_valid_generator, dg_test_generator, model_save_fn)

print(f'Saving model weights: {model_save_fn}')
torch.save(model.state_dict(), model_save_fn)

[2024-04-21 22:04:31]
Data generation ... 
Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54140.027  ,   274.65103], dtype=float32)
Coordinates:
  * level    (level) int32 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1100.1848  ,    5.642098], dtype=float32)
Coordinates:
  * level    (level) int32 500 500
Train model ... 
cuda
[  0/100] train_loss: 0.64616 valid_loss: 0.59500
[  1/100] train_loss: 0.57000 valid_loss: 0.58167
[  2/100] train_loss: 0.54771 valid_loss: 0.57684
[  3/100] train_loss: 0.53089 valid_loss: 0.57438
Early stopping
Saving model weights: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/PyTorch_NCI_tutorial_cnn_5d.h5
CPU times: user 7min 44s, sys: 45.3 s, total: 8min 29s
Wall time: 8min 23s


In [14]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
pred = train_nn_pytorch.create_predictions(model, dg_test_generator, 
                        mean=dg_train.mean.values, std=dg_train.std.values,
                        var_dict=dg_test.var_dict,
                        valid_time=dg_test.valid_time, 
                        lat=dg_test.ds.lat, 
                        lon=dg_test.ds.lon 
                       )
print(f'Saving predictions: {pred_save_fn}')
pred.to_netcdf(pred_save_fn)

evaluate(pred, iterative, test_years)

[2024-04-21 22:12:55]
cuda
torch.Size([17400, 2, 32, 64])
Saving predictions: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/PyTorch_NCI_tutorial_cnn_5d.nc
Evaluating forecast ...
all test_years: [2021, 2022]
load_test_data, var: z
load_test_data, var: t
<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 770.5
    t        float64 3.438
CPU times: user 20.5 s, sys: 3.99 s, total: 24.4 s
Wall time: 57.7 s


# fccnn_6h_iter

In [15]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )

print (60*"-")
batch_size = 32
lead_time = 6
iterative_lead_time = 5 * 24 
print ('save_prefix:', save_prefix)
model_save_fn=f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/saved_models/{save_prefix}_fccnn_6h_iter.h5'
pred_save_fn =f'/scratch/vp91/mah900/NCI-Weatherbench/pred_dir/{save_prefix}_fccnn_6h_iter.nc'
lr = 1e-4
iterative = True

[2024-04-21 22:13:53]
------------------------------------------------------------
save_prefix: PyTorch_NCI_tutorial
CPU times: user 50 µs, sys: 11 µs, total: 61 µs
Wall time: 53.9 µs


In [16]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
dg_train, dg_valid, dg_test = data_generate(ds, lead_time, batch_size,
                                           variables, train_years, valid_years, test_years)

params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 6}
dg_train_generator = torch.utils.data.DataLoader(dg_train, **params)
params['shuffle'] = False
dg_valid_generator = torch.utils.data.DataLoader(dg_valid, **params)
dg_test_generator = torch.utils.data.DataLoader(dg_test, **params)

model = train(dg_train_generator, dg_valid_generator, dg_test_generator, model_save_fn)

print(f'Saving model weights: {model_save_fn}')
torch.save(model.state_dict(), model_save_fn)

[2024-04-21 22:13:53]
Data generation ... 
Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54140.027  ,   274.65103], dtype=float32)
Coordinates:
  * level    (level) int32 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1100.1848  ,    5.642098], dtype=float32)
Coordinates:
  * level    (level) int32 500 500
Train model ... 
cuda
[  0/100] train_loss: 0.09661 valid_loss: 0.04356
[  1/100] train_loss: 0.03987 valid_loss: 0.03860
[  2/100] train_loss: 0.03604 valid_loss: 0.03561
[  3/100] train_loss: 0.03411 valid_loss: 0.03379
[  4/100] train_loss: 0.03285 valid_loss: 0.03475
[  5/100] train_loss: 0.03187 valid_loss: 0.03208
[  6/100] train_loss: 0.03107 valid_loss: 0.03095
[  7/100] train_loss: 0.03041 valid_loss: 0.03040
[  8/100] train_loss: 0.02984 valid_loss: 0.03017
[  9/100] train_loss: 0.02930 valid_loss: 0.03089
[ 10/100] train_loss: 0.02888 valid_loss: 0.02950
[ 11/100] train_loss: 0.02849 valid_loss: 0

In [17]:
%%time
print( f'[{datetime.now().replace(microsecond=0)}]' )
    
pred = train_nn_pytorch.create_iterative_predictions(model, dg_test_generator, 
                                        max_lead_time= 5*24,
                                        mean=dg_train.mean.values, std=dg_train.std.values,
                                        var_dict=dg_test.var_dict,
                                        valid_time=dg_test.valid_time, 
                                        lat=dg_test.ds.lat, 
                                        lon=dg_test.ds.lon,
                                        state=dg_test.data[:dg_test.n_samples], 
                                        lead_time=dg_test.lead_time, 
                                        init_time=dg_test.init_time
                                        )
print(f'Saving predictions: {pred_save_fn}')
pred.to_netcdf(pred_save_fn)

evaluate(pred, iterative, test_years)

[2024-04-21 23:03:05]
cuda
torch.Size([17514, 2, 32, 64])
Finish - loop 0
Finish - loop 1
Finish - loop 2
Finish - loop 3
Finish - loop 4
Finish - loop 5
Finish - loop 6
Finish - loop 7
Finish - loop 8
Finish - loop 9
Finish - loop 10
Finish - loop 11
Finish - loop 12
Finish - loop 13
Finish - loop 14
Finish - loop 15
Finish - loop 16
Finish - loop 17
Finish - loop 18
Finish - loop 19
(20, 17514, 2, 32, 64)
End prediction fn
Saving predictions: /scratch/vp91/mah900/NCI-Weatherbench/pred_dir/PyTorch_NCI_tutorial_fccnn_6h_iter.nc
Evaluating forecast ...
all test_years: [2021, 2022]
load_test_data, var: z
load_test_data, var: t
Merging ...
<xarray.Dataset>
Dimensions:    (lead_time: 20)
Coordinates:
  * lead_time  (lead_time) int64 6 12 18 24 30 36 42 ... 90 96 102 108 114 120
Data variables:
    z          (lead_time) float64 105.6 164.5 226.0 ... 1.089e+03 1.123e+03
    t          (lead_time) float64 1.097 1.45 1.679 1.87 ... 4.617 4.77 4.911
CPU times: user 2min 5s, sys: 44.5 s, total:

In [18]:
pred

<xarray.Dataset>
Dimensions:    (lead_time: 20, time: 17514, lat: 32, lon: 64)
Coordinates:
  * lead_time  (lead_time) int64 6 12 18 24 30 36 42 ... 90 96 102 108 114 120
  * time       (time) datetime64[ns] 2021-01-01 ... 2022-12-31T17:00:00
  * lat        (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon        (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Data variables:
    z          (lead_time, time, lat, lon) float32 4.86e+04 ... 5.118e+04
    t          (lead_time, time, lat, lon) float32 252.9 253.3 ... 261.2 260.4